In [6]:
import pandas as pd
df_train = pd.read_csv('../Data/train.csv')
df_misnoneTrain = df_train.fillna("no value")
df_misnoneTrain
X = df_misnoneTrain.drop(columns=['Personality'])
y = df_misnoneTrain['Personality']


In [9]:
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

# Prepare data
X = df_misnoneTrain.drop(columns=['Personality']).copy()
y = df_misnoneTrain['Personality']

# Convert all object columns to string type for OneHotEncoder compatibility
for col in X.select_dtypes(include=['object']).columns:
    X[col] = X[col].astype(str)

# Encode target variable
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 3),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 3),
        'eval_metric': 'logloss',
        'random_state': 42
    }

    model = XGBClassifier(**param)
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_encoded, cv=cv, scoring='accuracy')
    trial.set_user_attr('params', param)
    print(f"[Trial {trial.number}] value: {scores.mean():.6f}, params: {param}")
    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Best Params:", study.best_params)
print("Best CV Score:", study.best_value)

[I 2025-07-04 14:04:26,988] A new study created in memory with name: no-name-e6fcf547-3940-4285-92c5-c6124a4b9acd
[I 2025-07-04 14:04:29,310] Trial 0 finished with value: 0.9685814511357311 and parameters: {'n_estimators': 113, 'max_depth': 8, 'learning_rate': 0.05840816050844458, 'subsample': 0.707160657268785, 'colsample_bytree': 0.7574330025003639, 'gamma': 1.0757972851358337, 'reg_alpha': 2.594267817996071, 'reg_lambda': 2.4974716519912166}. Best is trial 0 with value: 0.9685814511357311.
[I 2025-07-04 14:04:29,310] Trial 0 finished with value: 0.9685814511357311 and parameters: {'n_estimators': 113, 'max_depth': 8, 'learning_rate': 0.05840816050844458, 'subsample': 0.707160657268785, 'colsample_bytree': 0.7574330025003639, 'gamma': 1.0757972851358337, 'reg_alpha': 2.594267817996071, 'reg_lambda': 2.4974716519912166}. Best is trial 0 with value: 0.9685814511357311.


[Trial 0] value: 0.968581, params: {'n_estimators': 113, 'max_depth': 8, 'learning_rate': 0.05840816050844458, 'subsample': 0.707160657268785, 'colsample_bytree': 0.7574330025003639, 'gamma': 1.0757972851358337, 'reg_alpha': 2.594267817996071, 'reg_lambda': 2.4974716519912166, 'eval_metric': 'logloss', 'random_state': 42}


[I 2025-07-04 14:04:31,337] Trial 1 finished with value: 0.9683115456026676 and parameters: {'n_estimators': 270, 'max_depth': 3, 'learning_rate': 0.10371201604274918, 'subsample': 0.7379356782947581, 'colsample_bytree': 0.8212803837238556, 'gamma': 3.1897843245164017, 'reg_alpha': 0.7744856970319051, 'reg_lambda': 1.4280727962341722}. Best is trial 0 with value: 0.9685814511357311.


[Trial 1] value: 0.968312, params: {'n_estimators': 270, 'max_depth': 3, 'learning_rate': 0.10371201604274918, 'subsample': 0.7379356782947581, 'colsample_bytree': 0.8212803837238556, 'gamma': 3.1897843245164017, 'reg_alpha': 0.7744856970319051, 'reg_lambda': 1.4280727962341722, 'eval_metric': 'logloss', 'random_state': 42}


[I 2025-07-04 14:04:36,923] Trial 2 finished with value: 0.9686354176686109 and parameters: {'n_estimators': 682, 'max_depth': 5, 'learning_rate': 0.022721008733383664, 'subsample': 0.7299243221454034, 'colsample_bytree': 0.741573423812497, 'gamma': 1.2252051644659683, 'reg_alpha': 2.422999534712565, 'reg_lambda': 2.162293216766288}. Best is trial 2 with value: 0.9686354176686109.


...

[Trial 47] value: 0.968419, params: {'n_estimators': 517, 'max_depth': 9, 'learning_rate': 0.11313596495219697, 'subsample': 0.6208495729969231, 'colsample_bytree': 0.733180876734072, 'gamma': 0.4509937942513198, 'reg_alpha': 2.7901679842928178, 'reg_lambda': 1.3001098305818148, 'eval_metric': 'logloss', 'random_state': 42}


[I 2025-07-04 14:07:52,324] Trial 48 finished with value: 0.9688513566687945 and parameters: {'n_estimators': 845, 'max_depth': 9, 'learning_rate': 0.010466391516671406, 'subsample': 0.6828234878032978, 'colsample_bytree': 0.6747834486086363, 'gamma': 2.226990760463745, 'reg_alpha': 2.7876067571648924, 'reg_lambda': 2.3046720773646947}. Best is trial 48 with value: 0.9688513566687945.


[Trial 48] value: 0.968851, params: {'n_estimators': 845, 'max_depth': 9, 'learning_rate': 0.010466391516671406, 'subsample': 0.6828234878032978, 'colsample_bytree': 0.6747834486086363, 'gamma': 2.226990760463745, 'reg_alpha': 2.7876067571648924, 'reg_lambda': 2.3046720773646947, 'eval_metric': 'logloss', 'random_state': 42}


[I 2025-07-04 14:07:59,072] Trial 49 finished with value: 0.9687973609884487 and parameters: {'n_estimators': 831, 'max_depth': 7, 'learning_rate': 0.012666791560954654, 'subsample': 0.681699728976839, 'colsample_bytree': 0.6531121686893251, 'gamma': 2.236766337544932, 'reg_alpha': 2.1547472409434425, 'reg_lambda': 2.5302382291846524}. Best is trial 48 with value: 0.9688513566687945.


[Trial 49] value: 0.968797, params: {'n_estimators': 831, 'max_depth': 7, 'learning_rate': 0.012666791560954654, 'subsample': 0.681699728976839, 'colsample_bytree': 0.6531121686893251, 'gamma': 2.236766337544932, 'reg_alpha': 2.1547472409434425, 'reg_lambda': 2.5302382291846524, 'eval_metric': 'logloss', 'random_state': 42}
Best Params: {'n_estimators': 845, 'max_depth': 9, 'learning_rate': 0.010466391516671406, 'subsample': 0.6828234878032978, 'colsample_bytree': 0.6747834486086363, 'gamma': 2.226990760463745, 'reg_alpha': 2.7876067571648924, 'reg_lambda': 2.3046720773646947}
Best CV Score: 0.9688513566687945


In [11]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from xgboost import XGBClassifier
import joblib

# Use df_misnoneTrain for modeling
df = df_misnoneTrain.copy()

# Convert all object columns to string type for OneHotEncoder compatibility
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype(str)

categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
if 'Personality' in categorical_cols:
    categorical_cols.remove('Personality')
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Encode target variable
le = LabelEncoder()
y_encoded = le.fit_transform(df['Personality'])

# Preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

# Create pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        n_estimators=845,
        max_depth=9,
        learning_rate=0.010466391516671406,
        subsample=0.6828234878032978,
        colsample_bytree=0.6747834486086363,
        gamma=2.226990760463745,
        reg_alpha=2.7876067571648924,
        reg_lambda=2.3046720773646947,
        eval_metric='logloss',
        random_state=42
    ))
])

# Fit pipeline
pipeline.fit(df.drop('Personality', axis=1), y_encoded)

# Save pipeline and label encoder
joblib.dump({'pipeline': pipeline, 'label_encoder': le}, 'final_model_pipeline_misnone.pkl')

['final_model_pipeline_misnone.pkl']

In [ ]:
import numpy as np
df_submission1 = pd.read_csv('../Data/test.csv')

for col in df_submission1.select_dtypes(include=['object']).columns:
    df_submission1[col] = df_submission1[col].astype(str)

df_submission1 = df_submission1.replace([np.nan, np.inf, -np.inf], 'no value')


model_bundle = joblib.load('final_model_pipeline_misnone.pkl')
pipeline = model_bundle['pipeline']
le = model_bundle['label_encoder']

preds_numeric = pipeline.predict(df_submission1)
preds_label = le.inverse_transform(preds_numeric)

submission = pd.DataFrame({
    'id': df_submission1['id'],
    'Personality': preds_label
})

submission.to_csv('submission1.csv', index=False)
submission.head()

,id,Personality
0,18524,Extrovert
1,18525,Introvert
2,18526,Extrovert
3,18527,Extrovert
4,18528,Introvert
